In [1]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import json

In [2]:
with open('APIkeys.json') as f:
    keys = json.load(f)
    omdbapi = keys['OMDBapi']

In [3]:
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+omdbapi

## Function for printing JSON dataset

In [4]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for k in list_keys:
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)

## Function to download poster

In [5]:
def save_poster(json_data):
    import os
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()

## Function to create/update the local movie database with the data retreived from the web


Saves the movie data (Title, Year, Runtime, Country, Metascore, and IMDB rating) into a local SQLite database called 'movies.sqlite'

In [6]:
#this is the query string
#http://www.omdbapi.com/?t=Toy+Story&y=1995

In [31]:
def save_in_database(json_data):
    
    #filename = input("Please enter a name for the database (extension not needed, it will be added automatically): ")
    #filename = filename+'.sqlite'
    filename = 'IMDB.sqlite'
    
    import sqlite3
    conn = sqlite3.connect(str(filename))
    cur=conn.cursor()
    
    title = json_data['Title']
    # Goes through the json dataset and extracts information if it is available
    if json_data['Year']!='N/A':
        year = json_data['Year']
    else:
        year='NA'
    if json_data['Rated']!='N/A':
        rated = json_data['Rated']
    else:
        rated='NA'
    if json_data['Released']!='N/A':
        released = json_data['Released']
    else:
        released='NA'
    if json_data['Runtime']!='N/A':
        runtime = int(json_data['Runtime'].split()[0])
    else:
        runtime=-1
    if json_data['Awards']!='N/A':
        awards = json_data['Awards']
    else:
        awards='NA'
    if json_data['Metascore']!='N/A':
        metascore = float(json_data['Metascore'])
    else:
        metascore=-1
    if json_data['imdbRating']!='N/A':
        imdb_rating = float(json_data['imdbRating'])
    else:
        imdb_rating=-1
    if json_data['imdbVotes']!='N/A':
        imdb_votes = json_data['imdbVotes']
    else:
        imdb_votes=-1
    if json_data['imdbID']!='N/A':
        imdbId = json_data['imdbID']
    
    
    # SQL commands
    cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfo 
    (Title TEXT, Year TEXT, Rated TEXT, Released TEXT, Runtime INTEGER, Awards TEXT, Metascore REAL, IMDBRating REAL, imdbVotes TEXT, imdbID TEXT)''')
    
    cur.execute('SELECT Title FROM MovieInfo WHERE Title = ? ', (title,))
    row = cur.fetchone()
    
    if row is None:
        cur.execute('''INSERT INTO MovieInfo (Title, Year, Rated, Released, Runtime, Awards, Metascore, IMDBRating, imdbVotes, imdbID )
                VALUES (?,?,?,?,?,?,?,?,?,?)''', (title,year,rated,released,runtime,awards,metascore,imdb_rating,imdb_votes,imdbId))
    else:
        print("Record already found. No update made.")
    
    # Commits the change and close the connection to the database
    conn.commit()
    conn.close()

## Function to print contents of the local database

In [8]:
def print_database(database):
    
    import sqlite3
    conn = sqlite3.connect(str(database))
    cur=conn.cursor()
    
    for row in cur.execute('SELECT * FROM MovieInfo'):
        print(row)
    conn.close()

## Function to save the database content in an Excel file

In [9]:
def save_in_excel(filename, database):
    
    if filename.split('.')[-1]!='xls' and filename.split('.')[-1]!='xlsx':
        print ("Filename does not have correct extension. Please try again")
        return None
    
    import pandas as pd
    import sqlite3
    
    conn = sqlite3.connect(str(database))
    #cur=conn.cursor()
    
    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)
    conn.close()
    
    df.to_excel(filename,sheet_name='Movie Info')

## Function to search for information about a movie

In [10]:
def search_movie(title,year):
    if len(title) < 1 or title=='quit': 
        print("Goodbye now...")
        return None

    try:
        url = serviceurl + urllib.parse.urlencode({'t': title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
        
        print(f'Retrieving the data of "{title}" now... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_in_database(json_data)
            
        else:
            print("Error encountered: ",json_data['Error'])
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")

## Load Disney titles to get all the information through function search_movie

In [29]:
#Load Disney titles

disney_df = pd.read_csv('disney_movies_total_gross.csv')
disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,year
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",1937
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",1940
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000","$2,187,090,808",1940
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000","$1,078,510,579",1946
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000","$920,608,730",1950


In [30]:
#test with this df to see if it works as free API call is only 1000 calls per day
test_df = disney_df.loc[disney_df['movie_title'].isin(['Change for the Oceans','Pinocchio'])]
test_df

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,year
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",1940


In [14]:
#query for all the movies in our disney list and get the latest information from IMDB

for i in disney_df.index: 
    title = disney_df['movie_title'][i]
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)

Snow White and the Seven Dwarfs
1937
Retrieving the data of "Snow White and the Seven Dwarfs" now... 
--------------------------------------------------
Title: Snow White and the Seven Dwarfs
Year: 1937
Rated: Approved
Released: 04 Feb 1938
Runtime: 83 min
Genre: Animation, Adventure, Family
Director: William Cottrell, David Hand, Wilfred Jackson
Writer: Jacob Grimm, Wilhelm Grimm, Ted Sears
Actors: Adriana Caselotti, Harry Stockwell, Lucille La Verne
Plot: Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.
Language: English
Country: United States
Awards: Won 1 Oscar. 11 wins & 6 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Metacritic', 'Value': '95/100'}]
Metascore: 95
imdbRating: 7.6
imdbVotes: 194,079
imdbID: tt0029583
--------------------------------------------------
Record already found. No update made.
Pinocchio
1940
Retrieving the data o

--------------------------------------------------
Title: Babes in Toyland
Year: 1961
Rated: Approved
Released: 14 Dec 1961
Runtime: 106 min
Genre: Comedy, Family, Fantasy
Director: Jack Donohue
Writer: Victor Herbert, Glen MacDonough, Ward Kimball
Actors: Ray Bolger, Tommy Sands, Annette Funicello
Plot: Mary Contrary is set to marry Tom Piper when he is kidnapped by Roderigo and Gonzorgo, two goons working for the evil Barnaby who wants to marry Mary for her inheritance.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 1 win & 6 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.2/10'}, {'Source': 'Rotten Tomatoes', 'Value': '36%'}, {'Source': 'Metacritic', 'Value': '55/100'}]
Metascore: 55
imdbRating: 6.2
imdbVotes: 4,118
imdbID: tt0054649
--------------------------------------------------
Record already found. No update made.
Bon Voyage!
1962
Retrieving the data of "Bon Voyage!" now... 
----------------------------------------------

--------------------------------------------------
Title: The Many Adventures of Winnie the Pooh
Year: 1977
Rated: G
Released: 11 Mar 1977
Runtime: 74 min
Genre: Animation, Adventure, Comedy
Director: John Lounsbery, Wolfgang Reitherman, Ben Sharpsteen
Writer: A.A. Milne, Larry Clemmons, Ralph Wright
Actors: Sebastian Cabot, Junius Matthews, Barbara Luddy
Plot: In this collection of animated shorts based on the stories and characters by A.A. Milne, Winnie the Pooh, a honey-loving teddy bear, embarks on some eccentric adventures.
Language: English
Country: United States
Awards: 1 nomination
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '100%'}]
Metascore: N/A
imdbRating: 7.5
imdbVotes: 35,522
imdbID: tt0076363
--------------------------------------------------
Record already found. No update made.
The Rescuers
1977
Retrieving the data of "The Rescuers" now... 
--------------------------------------------------
Title: The Rescu

--------------------------------------------------
Title: TRON
Year: 1982
Rated: PG
Released: 09 Jul 1982
Runtime: 96 min
Genre: Action, Adventure, Sci-Fi
Director: Steven Lisberger
Writer: Steven Lisberger, Bonnie MacBird, Charles S. Haas
Actors: Jeff Bridges, Bruce Boxleitner, David Warner
Plot: A computer hacker is abducted into the digital world and forced to participate in gladiatorial games where his only chance of escape is with the help of a heroic security program.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 2 wins & 8 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '71%'}, {'Source': 'Metacritic', 'Value': '58/100'}]
Metascore: 58
imdbRating: 6.8
imdbVotes: 118,943
imdbID: tt0084827
--------------------------------------------------
Record already found. No update made.
Tex
1982
Retrieving the data of "Tex" now... 
--------------------------------------------------
Title:

--------------------------------------------------
Title: My Science Project
Year: 1985
Rated: PG
Released: 09 Aug 1985
Runtime: 94 min
Genre: Action, Adventure, Comedy
Director: Jonathan R. Betuel
Writer: Jonathan R. Betuel
Actors: John Stockwell, Danielle von Zerneck, Fisher Stevens
Plot: A high school student and his friend find a strange, orb-like piece of equipment to use as a science project, but must stop it when it begins to threaten mankind.
Language: English, Latin
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '11%'}, {'Source': 'Metacritic', 'Value': '38/100'}]
Metascore: 38
imdbRating: 6.1
imdbVotes: 5,777
imdbID: tt0089652
--------------------------------------------------
Record already found. No update made.
The Journey of Natty Gann
1985
Retrieving the data of "The Journey of Natty Gann" now... 
--------------------------------------------------
Title: The Journey of Natty Ga

Record already found. No update made.
The Color of Money
1986
Retrieving the data of "The Color of Money" now... 
--------------------------------------------------
Title: The Color of Money
Year: 1986
Rated: R
Released: 17 Oct 1986
Runtime: 119 min
Genre: Drama, Sport
Director: Martin Scorsese
Writer: Walter Tevis, Richard Price
Actors: Paul Newman, Tom Cruise, Mary Elizabeth Mastrantonio
Plot: Fast Eddie Felson teaches a cocky but immensely talented protégé the ropes of pool hustling, which in turn inspires him to make an unlikely comeback.
Language: English, Spanish
Country: United States
Awards: Won 1 Oscar. 3 wins & 10 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.0/10'}, {'Source': 'Rotten Tomatoes', 'Value': '89%'}, {'Source': 'Metacritic', 'Value': '77/100'}]
Metascore: 77
imdbRating: 7.0
imdbVotes: 81,054
imdbID: tt0090863
--------------------------------------------------
Record already found. No update made.
Outrageous Fortune
1987
Retrieving 

--------------------------------------------------
Title: Good Morning, Vietnam
Year: 1987
Rated: R
Released: 15 Jan 1988
Runtime: 121 min
Genre: Biography, Comedy, Drama
Director: Barry Levinson
Writer: Mitch Markowitz
Actors: Robin Williams, Forest Whitaker, Tom. T. Tran
Plot: In 1965, an unorthodox and irreverent DJ named Adrian Cronauer begins to shake up things when he is assigned to the U.S. Armed Services radio station in Vietnam.
Language: English
Country: United States
Awards: Nominated for 1 Oscar. 7 wins & 4 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '91%'}, {'Source': 'Metacritic', 'Value': '67/100'}]
Metascore: 67
imdbRating: 7.3
imdbVotes: 136,423
imdbID: tt0093105
--------------------------------------------------
Record already found. No update made.
Shoot to Kill
1988
Retrieving the data of "Shoot to Kill" now... 
--------------------------------------------------
Title: Shoot to Kill
Yea

--------------------------------------------------
Title: Ernest Saves Christmas
Year: 1988
Rated: PG
Released: 11 Nov 1988
Runtime: 95 min
Genre: Comedy, Family, Fantasy
Director: John R. Cherry III
Writer: Ed Turner, B. Kline
Actors: Jim Varney, Douglas Seale, Oliver Clark
Plot: Ernest helps Santa Claus as he searches for his successor.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '36%'}, {'Source': 'Metacritic', 'Value': '44/100'}]
Metascore: 44
imdbRating: 5.7
imdbVotes: 11,383
imdbID: tt0095107
--------------------------------------------------
Record already found. No update made.
Oliver & Company
1988
Retrieving the data of "Oliver & Company" now... 
--------------------------------------------------
Title: Oliver & Company
Year: 1988
Rated: G
Released: 18 Nov 1988
Runtime: 74 min
Genre: Animation, Adventure, Comedy
Director: George Scribner
Writer: Jim Cox, Tim Dis

KeyboardInterrupt: 

In [32]:
# #Resolving issues with the parameters and starting from where we stopped and continue to the end of list

# for i in range(455,len(disney_df)): 
#     title = disney_df['movie_title'][i]
#     year = disney_df['year'][i]
#     print (disney_df['movie_title'][i])
#     print (disney_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)

The Hoax
2007
Retrieving the data of "The Hoax" now... 
--------------------------------------------------
Title: The Hoax
Year: 2007
Rated: Not Rated
Released: 01 Feb 2007
Runtime: 99 min
Genre: Thriller
Director: Huw Bowen
Writer: Walker O'Brien
Actors: Gene Ganssle, Lynne Pyrce, Kathleen Benner, Brian Bozick
Plot: A psychiatrist's wife is gunned down by her husband's crazed patient outside a fine restaurant. In his efforts to debunk the delusions that drove his patient's violent attack, Dr. David ...
Language: English
Country: USA
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '4.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '86%'}]
Metascore: N/A
imdbRating: 4.3
imdbVotes: 13
imdbID: tt1266072
--------------------------------------------------
Record already found. No update made.
The Invisible
2007
Retrieving the data of "The Invisible" now... 
--------------------------------------------------
Title: The Invisible
Year: 2007
Rated: PG-13
Released: 27 Apr 20

--------------------------------------------------
Title: The Chronicles of Narnia: Prince Caspian
Year: 2008
Rated: PG
Released: 16 May 2008
Runtime: 150 min
Genre: Action, Adventure, Family
Director: Andrew Adamson
Writer: Andrew Adamson, Christopher Markus, Stephen McFeely
Actors: Ben Barnes, Skandar Keynes, Georgie Henley
Plot: The Pevensie siblings return to Narnia, where they are enlisted to once again help ward off an evil king and restore the rightful heir to the land's throne, Prince Caspian.
Language: English
Country: United States, Poland, Slovenia, Czech Republic, United Kingdom
Awards: 4 wins & 21 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '66%'}, {'Source': 'Metacritic', 'Value': '62/100'}]
Metascore: 62
imdbRating: 6.5
imdbVotes: 206,153
imdbID: tt0499448
--------------------------------------------------
Record already found. No update made.
WALL-E
2008
Retrieving the data of "WALL-E" now... 
--

--------------------------------------------------
Title: Confessions of a Shopaholic
Year: 2009
Rated: PG
Released: 13 Feb 2009
Runtime: 104 min
Genre: Comedy, Romance
Director: P.J. Hogan
Writer: Tracey Jackson, Tim Firth, Kayla Alpert
Actors: Isla Fisher, Hugh Dancy, Krysten Ritter
Plot: A college grad lands a job as a financial journalist in New York City to support where she nurtures her shopping addiction and falls for a wealthy entrepreneur.
Language: English, Finnish, Spanish, French
Country: United States
Awards: 1 win & 3 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.9/10'}, {'Source': 'Rotten Tomatoes', 'Value': '27%'}, {'Source': 'Metacritic', 'Value': '38/100'}]
Metascore: 38
imdbRating: 5.9
imdbVotes: 77,344
imdbID: tt1093908
--------------------------------------------------
Record already found. No update made.
Jonas Brothers: The 3D Concert Experi…
2009
Retrieving the data of "Jonas Brothers: The 3D Concert Experi…" now... 
Error encountered: 

--------------------------------------------------
Title: The Princess and the Frog
Year: 2009
Rated: G
Released: 11 Dec 2009
Runtime: 97 min
Genre: Animation, Adventure, Comedy
Director: Ron Clements, John Musker
Writer: Ron Clements, John Musker, Greg Erb
Actors: Anika Noni Rose, Keith David, Oprah Winfrey
Plot: A waitress, desperate to fulfill her dreams as a restaurant owner, is set on a journey to turn a frog prince back into a human being, but she has to face the same problem after she kisses him.
Language: English, French
Country: United States
Awards: Nominated for 3 Oscars. 10 wins & 42 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '85%'}, {'Source': 'Metacritic', 'Value': '73/100'}]
Metascore: 73
imdbRating: 7.1
imdbVotes: 137,228
imdbID: tt0780521
--------------------------------------------------
Record already found. No update made.
Old Dogs
2009
Retrieving the data of "Old Dogs" now... 
-------

--------------------------------------------------
Title: Secretariat
Year: 2010
Rated: PG
Released: 08 Oct 2010
Runtime: 123 min
Genre: Biography, Drama, Family
Director: Randall Wallace
Writer: Mike Rich, William Nack
Actors: Diane Lane, John Malkovich, Margo Martindale
Plot: Penny Chenery Tweedy and colleagues guide her long-shot but precocious stallion to set, in 1973, the unbeaten record for winning the Triple Crown.
Language: English
Country: United States
Awards: 3 wins & 6 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '61/100'}]
Metascore: 61
imdbRating: 7.2
imdbVotes: 28,112
imdbID: tt1028576
--------------------------------------------------
Tangled
2010
Retrieving the data of "Tangled" now... 
--------------------------------------------------
Title: Tangled
Year: 2010
Rated: PG
Released: 24 Nov 2010
Runtime: 100 min
Genre: Animation, Adventure, Comedy
Director:

...

Error encountered:  Movie not found!
Frozen
2013
Retrieving the data of "Frozen" now... 
--------------------------------------------------
Title: Frozen
Year: 2013
Rated: PG
Released: 27 Nov 2013
Runtime: 102 min
Genre: Animation, Adventure, Comedy
Director: Chris Buck, Jennifer Lee
Writer: Jennifer Lee, Hans Christian Andersen, Chris Buck
Actors: Kristen Bell, Idina Menzel, Jonathan Groff
Plot: When the newly crowned Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter, her sister Anna teams up with a mountain man, his playful reindeer, and a snowman to change the weather condi
Language: English, Norwegian
Country: United States
Awards: Won 2 Oscars. 82 wins & 60 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '90%'}, {'Source': 'Metacritic', 'Value': '75/100'}]
Metascore: 75
imdbRating: 7.5
imdbVotes: 607,860
imdbID: tt2294629
----------------------------------

Error encountered:  Movie not found!
Guardians of the Galaxy
2014
Retrieving the data of "Guardians of the Galaxy" now... 
--------------------------------------------------
Title: Guardians of the Galaxy
Year: 2014
Rated: PG-13
Released: 01 Aug 2014
Runtime: 121 min
Genre: Action, Adventure, Comedy
Director: James Gunn
Writer: James Gunn, Nicole Perlman, Dan Abnett
Actors: Chris Pratt, Vin Diesel, Bradley Cooper
Plot: A group of intergalactic criminals must pull together to stop a fanatical warrior with plans to purge the universe.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 52 wins & 102 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '8.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '76/100'}]
Metascore: 76
imdbRating: 8.1
imdbVotes: 1,131,146
imdbID: tt2015381
--------------------------------------------------
The Hundred-Foot Journey
2014
Retrieving the data of "The Hundred-Foot Journ

Tomorrowland
2015
Retrieving the data of "Tomorrowland" now... 
--------------------------------------------------
Title: Tomorrowland
Year: 2015
Rated: PG
Released: 22 May 2015
Runtime: 130 min
Genre: Action, Adventure, Family
Director: Brad Bird
Writer: Damon Lindelof, Brad Bird, Jeff Jensen
Actors: George Clooney, Britt Robertson, Hugh Laurie
Plot: Bound by a shared destiny, a teen bursting with scientific curiosity and a former boy-genius inventor embark on a mission to unearth the secrets of a place somewhere in time and space that exists in their collective memory.
Language: English, French, Japanese
Country: United States, Spain, France, Canada, United Kingdom
Awards: 2 wins & 19 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '50%'}, {'Source': 'Metacritic', 'Value': '60/100'}]
Metascore: 60
imdbRating: 6.4
imdbVotes: 179,774
imdbID: tt1964418
--------------------------------------------------
Inside Out
201

--------------------------------------------------
Title: Captain America: Civil War
Year: 2016
Rated: PG-13
Released: 06 May 2016
Runtime: 147 min
Genre: Action, Adventure, Sci-Fi
Director: Anthony Russo, Joe Russo
Writer: Christopher Markus, Stephen McFeely, Joe Simon
Actors: Chris Evans, Robert Downey Jr., Scarlett Johansson
Plot: Political involvement in the Avengers' affairs causes a rift between Captain America and Iron Man.
Language: English, German, Xhosa, Russian, Romanian, Hindi
Country: United States
Awards: 16 wins & 73 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '90%'}, {'Source': 'Metacritic', 'Value': '75/100'}]
Metascore: 75
imdbRating: 7.8
imdbVotes: 745,441
imdbID: tt3498820
--------------------------------------------------
Alice Through the Looking Glass
2016
Retrieving the data of "Alice Through the Looking Glass" now... 
--------------------------------------------------
Title: Alice Throug

--------------------------------------------------
Title: Rogue One: A Star Wars Story
Year: 2016
Rated: PG-13
Released: 16 Dec 2016
Runtime: 133 min
Genre: Action, Adventure, Sci-Fi
Director: Gareth Edwards
Writer: Chris Weitz, Tony Gilroy, John Knoll
Actors: Felicity Jones, Diego Luna, Alan Tudyk
Plot: In a time of conflict, a group of unlikely heroes band together on a mission to steal the plans to the Death Star, the Empire's ultimate weapon of destruction.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 24 wins & 85 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '84%'}, {'Source': 'Metacritic', 'Value': '65/100'}]
Metascore: 65
imdbRating: 7.8
imdbVotes: 597,163
imdbID: tt3748528
--------------------------------------------------


In [ ]:
# #if we query each movie title & year manually

# title = input('\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): ')
# if len(title) < 1 or title=='quit': 
#     print("Goodbye now...")
# year = input('\nEnter the year of a movie (enter \'quit\' or hit ENTER to quit): ')
# if len(year) < 1 or year=='quit': 
#     print("Goodbye now...")
# else:
#     search_movie(title,year)

In [ ]:
# #Show downloaded poster of Titanic
# from IPython.display import Image
# Image("Posters/Toy Story.jpg")

## Print the content of the local database

In [33]:
print_database('IMDB.sqlite')

('Pinocchio', 1940, 'G', '23 Feb 1940', 88, 'Won 2 Oscars. 7 wins total', 99.0, 7.4, '140,877', 'tt0032910')
("Blackbeard's Ghost", 1968, 'Approved', '08 Feb 1968', 106, 'NA', 71.0, 6.8, '5,906', 'tt0062737')
('Snow White and the Seven Dwarfs', 1937, 'Approved', '04 Feb 1938', 83, 'Won 1 Oscar. 11 wins & 6 nominations total', 95.0, 7.6, '194,079', 'tt0029583')
('Fantasia', 1940, 'G', '19 Sep 1941', 125, 'Won 2 Oscars. 8 wins & 1 nomination total', 96.0, 7.7, '94,045', 'tt0032455')
('Song of the South', 1946, 'G', '20 Nov 1946', 94, 'Won 2 Oscars. 2 wins & 1 nomination total', 54.0, 7.1, '13,653', 'tt0038969')
('Cinderella', 1950, 'G', '04 Mar 1950', 74, 'Nominated for 3 Oscars. 6 wins & 7 nominations total', 85.0, 7.3, '155,521', 'tt0042332')
('20,000 Leagues Under the Sea', 1954, 'G', '20 Jul 1955', 127, 'Won 2 Oscars. 5 wins & 3 nominations total', 83.0, 7.2, '33,019', 'tt0046672')
('Lady and the Tramp', 1955, 'G', '22 Jun 1955', 76, 'Nominated for 1 BAFTA Film Award1 win & 2 nominat

## Save the database content into an Excel file

In [35]:
save_in_excel('IMDB.xlsx','IMDB.sqlite')

In [36]:
df=pd.read_excel('IMDB.xlsx')
df

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,0,Pinocchio,1940,G,23 Feb 1940,88,Won 2 Oscars. 7 wins total,99,7.4,"140,877",tt0032910
1,1,Blackbeard's Ghost,1968,Approved,08 Feb 1968,106,NaN,71,6.8,"5,906",tt0062737
2,2,Snow White and the Seven Dwarfs,1937,Approved,04 Feb 1938,83,Won 1 Oscar. 11 wins & 6 nominations total,95,7.6,"194,079",tt0029583
3,3,Fantasia,1940,G,19 Sep 1941,125,Won 2 Oscars. 8 wins & 1 nomination total,96,7.7,"94,045",tt0032455
4,4,Song of the South,1946,G,20 Nov 1946,94,Won 2 Oscars. 2 wins & 1 nomination total,54,7.1,"13,653",tt0038969
...,...,...,...,...,...,...,...,...,...,...,...
517,517,The Light Between Oceans,2016,PG-13,02 Sep 2016,133,1 win & 16 nominations,60,7.2,"55,855",tt2547584
518,518,Queen of Katwe,2016,PG,30 Sep 2016,124,3 wins & 26 nominations,73,7.4,"17,080",tt4341582
519,519,Doctor Strange,2016,PG-13,04 Nov 2016,115,Nominated for 1 Oscar. 20 wins & 68 nomination...,72,7.5,"685,849",tt1211837
520,520,Moana,2016,PG,23 Nov 2016,107,Nominated for 2 Oscars. 21 wins & 90 nominatio...,81,7.6,"315,408",tt3521164


In [37]:
# Save the database content into an CSV file
df.to_csv(r'IMDB_disney_movies.csv', index = None, header=True) 

In [38]:
#check the CSV is exported correctly
disney_df = pd.read_csv('IMDB_disney_movies.csv')
disney_df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,0,Pinocchio,1940,G,23 Feb 1940,88,Won 2 Oscars. 7 wins total,99,7.4,"140,877",tt0032910
1,1,Blackbeard's Ghost,1968,Approved,08 Feb 1968,106,NaN,71,6.8,"5,906",tt0062737
2,2,Snow White and the Seven Dwarfs,1937,Approved,04 Feb 1938,83,Won 1 Oscar. 11 wins & 6 nominations total,95,7.6,"194,079",tt0029583
3,3,Fantasia,1940,G,19 Sep 1941,125,Won 2 Oscars. 8 wins & 1 nomination total,96,7.7,"94,045",tt0032455
4,4,Song of the South,1946,G,20 Nov 1946,94,Won 2 Oscars. 2 wins & 1 nomination total,54,7.1,"13,653",tt0038969
